In [1]:
import math
import os
from os.path import isfile, join
import glob
import cv2
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
# Get all gt and sys files


gt_files = sorted(glob.glob('DJI_0004/gt_0004_pixel/*.txt'))
sys_files = sorted(glob.glob('DJI_0004/yolo_sort_tracking/*.txt'))

In [3]:
def calculate_iou(box1, box2):
    """
    Calculate Intersection over Union (IoU) between two bounding boxes.

    Args:
    - box1: A tuple representing the coordinates (x1, y1, x2, y2) of the first bounding box.
    - box2: A tuple representing the coordinates (x1, y1, x2, y2) of the second bounding box.

    Returns:
    - The IoU (Intersection over Union) value between the two bounding boxes.
    """
    x1, y1, x2, y2 = box1
    x3, y3, x4, y4 = box2

    intersection_area = max(0, min(x2, x4) - max(x1, x3)) * max(0, min(y2, y4) - max(y1, y3))
    area1 = (x2 - x1) * (y2 - y1)
    area2 = (x4 - x3) * (y4 - y3)
    union_area = area1 + area2 - intersection_area

    iou = intersection_area / union_area
    return iou

In [5]:
SYS=[]
GT=[]
for i in range(len(gt_files)):
    with open(gt_files[i], 'r') as f:
        gt_annotations = [tuple(map(float, line.strip().split())) for line in f.readlines()]
        GT.append(gt_annotations)
    with open(sys_files[i], 'r') as f:
        sys_annotations = [tuple(map(float, line.strip().split())) for line in f.readlines()]
    new_sys=[]
    for (k, gt) in enumerate(gt_annotations):
        gt_bbox = gt[2:]
        # print(gt_bbox)
        for (j,sys) in enumerate(sys_annotations):
            sys_bbox = sys[2:]
            # new_sys=[]
            iou = calculate_iou(gt_bbox, sys_bbox)
            if iou >= 0.5:
                # print(f'Matches Found in File {i}\n\t {k} in GT with {j} in SYS')
                new_sys.append(sys)
    
    SYS.append(new_sys)
    

In [77]:
GT[0]

[(0.0, 0.0, 191.0, 1525.0, 416.0, 1642.0),
 (1.0, 0.0, 1577.0, 957.0, 1664.0, 1123.0),
 (2.0, 1.0, 1649.0, 986.0, 1717.0, 1070.0),
 (3.0, 0.0, 0.0, 1665.0, 117.0, 1746.0)]

In [78]:
def compute_metrics(gt_list, sys_list):
    """
    Compute multi-object tracking metrics: FP, FN, IDS, MOTA
    """
    FP = 0  # False Positives
    FN = 0  # False Negatives
    IDS = 0  # ID Switches
    total_objects = 0  # Total number of objects
    total_iou = 0
    total_matches = 0
    distance=[]
    adults=0
    calfs = 0
    
    
    for frame_gt, frame_sys in zip(gt_list, sys_list):
        gt_ids = [obj[0] for obj in frame_gt]
        sys_ids = [obj[0] for obj in frame_sys]
        
        gt_cls = [obj[1] for obj in frame_gt]
        sys_cls = [obj[1] for obj in frame_sys] 
        
        for cls in gt_cls:
            if cls == 0:
                adults+=1
            else:
                calfs+=1
        total_objects += len(gt_ids)
        # print(f"Total of {total_objects} in Frame {frame_gt}")
        # for obj_id in gt_ids:
        #     if obj_id not in sys_ids:
        #         FN += 1
        
        # for obj_id in sys_ids:
        #     if obj_id not in gt_ids:
        #         FP += 1
        if len(frame_gt) > len(frame_sys):
            FN += len(frame_gt) - len(frame_sys)
    
        if len(frame_gt) < len(frame_sys):
            FP += len(frame_sys) - len(frame_gt)
        
        
        for (i,obj_gt), (j,obj_sys) in zip(enumerate(frame_gt), enumerate(frame_sys)):
            obj_id_gt = obj_gt[0]
            bbox_gt = obj_gt[2:]
            obj_id_sys = obj_sys[0]
            bbox_sys = obj_sys[2:]
            iou = calculate_iou(bbox_gt, bbox_sys)
                
            if obj_id_gt != obj_id_sys:
                IDS+=1
                if iou > 0.5:
                    total_matches+=1
                    d1 = 1-iou
                    distance.append(d1)
                    obj_id_sys = obj_id_gt
                    # IDS-=1
                    
            else:
                total_matches+=1
                d1 = 1-iou
                distance.append(d1)
                    
    MOTP = 1 - sum(distance) / total_matches if total_matches else 0
    MOTA = 1 - (FP + FN + IDS) / total_objects if total_objects else 0
    
    return FP, FN, IDS, MOTA, MOTP, total_objects, distance,adults,calfs

In [79]:
fp, fn, ids, mota, motp, t, d, adults, calfs = compute_metrics(GT,SYS)

In [80]:
adults

132

In [82]:
calfs

49

In [83]:
adults+calfs

181

In [62]:
def compute_metrics_post(gt_list, sys_list):
    """
    Compute multi-object tracking metrics: FP, FN, IDS, MOTA
    """
    FP = 0  # False Positives
    FN = 0  # False Negatives
    IDS = 0  # ID Switches
    total_objects = 0  # Total number of objects
    total_iou = 0
    total_matches = 0
    distance=[]
    
    
    for frame_gt, frame_sys in zip(gt_list, sys_list):
        gt_ids = [obj[0] for obj in frame_gt]
        sys_ids = [obj[0] for obj in frame_sys]
        
        total_objects += len(gt_ids)
        
        if len(frame_gt) > len(frame_sys):
            FN += len(frame_gt) - len(frame_sys)
    
        if len(frame_gt) < len(frame_sys):
            FP += len(frame_sys) - len(frame_gt)

        for (i,obj_gt), (j,obj_sys) in zip(enumerate(frame_gt), enumerate(frame_sys)):
            obj_id_gt = obj_gt[0]
            bbox_gt = obj_gt[2:]
            obj_id_sys = obj_sys[0]
            bbox_sys = obj_sys[2:]
            iou = calculate_iou(bbox_gt, bbox_sys)
                
            if obj_id_gt != obj_id_sys:
                IDS+=1
                if iou > 0.5:
                    total_matches+=1
                    d1 = 1-iou
                    distance.append(d1)
                    obj_id_sys = obj_id_gt
                    IDS-=1
                    
            else:
                total_matches+=1
                d1 = 1-iou
                distance.append(d1)
                    
    MOTP = 1 - sum(distance) / total_matches if total_matches else 0
    MOTA = 1 - (FP + FN + IDS) / total_objects if total_objects else 0
    
    return FP, FN, IDS, MOTA, MOTP, total_objects, distance

In [68]:
fpp, fnp, idsp, motap, motpp, tp, dp = compute_metrics_post(GT,SYS)

In [86]:
motap

0.7569060773480663

In [85]:
fnp

23

In [73]:
mota

0.0

In [74]:
motpp

0.739905312341764

In [85]:
def calculate_distance(box1, box2):
    """
    Calculate the distance between the centers of two bounding boxes.

    Args:
    - box1: A tuple representing the coordinates (x1, y1, x2, y2) of the first bounding box.
    - box2: A tuple representing the coordinates (x1, y1, x2, y2) of the second bounding box.

    Returns:
    - The distance between the centers of the two bounding boxes.
    """
    x1, y1, x2, y2 = box1
    x3, y3, x4, y4 = box2

    center_x1 = (x1 + x2) / 2
    center_y1 = (y1 + y2) / 2

    center_x2 = (x3 + x4) / 2
    center_y2 = (y3 + y4) / 2

    c1 = (center_x1, center_y1)
    c2 = (center_x2, center_y2)

    dc1c2 = euclidean_distance(c1,c2)


    return dc1c2


def find_bbox_center(x1, y1, x2, y2):
    """
    Calculate the center of a bounding box.

    Args:
    - x1, y1: The top-left coordinates of the bounding box.
    - x2, y2: The bottom-right coordinates of the bounding box.

    Returns:
    - A tuple representing the (x, y) coordinates of the center of the bounding box.
    """
    
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2
    return center_x, center_y
def euclidean_distance(point1, point2):
    """
    Calculate the Euclidean distance between two points.
    
    Args:
    - point1: A tuple representing the (x, y) coordinates of the first point.
    - point2: A tuple representing the (x, y) coordinates of the second point.
    
    Returns:
    - The Euclidean distance between the two points.
    """
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

In [86]:
calculate_distance(GT[0][0][2:], SYS[0][0][2:])

10.002229432382967

In [82]:
GT[0][0]

(0.0, 0.0, 191.0, 1525.0, 416.0, 1642.0)